### Get data from gaia
SELECT ALL gaia_source.source_id,gaia_source.ra,gaia_source.ra_error,gaia_source.dec,gaia_source.dec_error,gaia_source.pmra,gaia_source.pmra_error,gaia_source.pmdec,gaia_source.pmdec_error,gaia_source.phot_bp_mean_mag,gaia_source.radial_velocity,gaia_source.radial_velocity_error
FROM gaiadr2.gaia_source 
WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',183.721,12.785,3))=1  AND  (gaiadr2.gaia_source.phot_bp_mean_mag<=17)

In [3]:
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [9]:
job = Gaia.launch_job_async("SELECT ALL gaia_source.source_id,gaia_source.ra,gaia_source.ra_error,gaia_source.dec,gaia_source.dec_error,gaia_source.pmra,gaia_source.pmra_error,gaia_source.pmdec,gaia_source.pmdec_error,gaia_source.phot_bp_mean_mag,gaia_source.radial_velocity,gaia_source.radial_velocity_error FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',183.721,12.785,3))=1  AND  (gaiadr2.gaia_source.phot_bp_mean_mag<=17)")

Query finished.


In [10]:
r = job.get_results()
print(r['ra','dec'])

        ra                dec        
       deg                deg        
------------------ ------------------
182.06770195328755  11.79359280719235
181.25441451210122 11.008729614018321
181.53595754136217 11.184619989802899
181.99360808226916 11.587464938436003
181.48408156403372  11.31517732538975
182.04456959710964 11.780362569798461
181.59843013913613 11.175090701328484
182.54767299559865 12.083720544056007
 181.5548019301534 11.873944575898046
  182.013979921151 11.817131925823952
               ...                ...
 185.1716258720525 15.143946296280525
 185.2083723033049  15.27356951100647
185.23546139985038  15.30806853903222
185.30742776507591  15.31761270860593
185.10192943238832 15.143515510864267
185.29110244593113 15.361113183055878
184.70607105903395  15.43585725415703
 184.9304420448486 15.204390789719593
184.96258573048792 15.394453952181477
184.70304909052405  15.48668823506751
Length = 9514 rows
